In [1]:
import pandas as pd
import numpy as np

In [103]:
df = pd.read_csv("/Users/agoraqa/Desktop/videos/iOS/test3_blackmagic/point/loss10_1_point.csv")
# df.loc[(df['PIX_VALUE_DIFF'] <= 100) & (df['PIX_VALUE_DIFF'] >= -100), 'PIX_VALUE_DIFF'] = 0
print(df)

       FRAME  PIX_VALUE_DIFF
0        1.0               0
1        2.0             975
2        3.0               8
3        4.0              10
4        5.0              15
5        6.0             577
6        7.0               2
7        8.0              17
8        9.0               9
9       10.0               8
10      11.0             496
11      12.0              21
12      13.0               9
13      14.0               4
14      15.0             373
15      16.0               7
16      17.0               5
17      18.0               1
18      19.0             266
19      20.0              15
20      21.0              17
21      22.0              14
22      23.0              11
23      24.0               6
24      25.0             225
25      26.0               6
26      27.0               5
27      28.0               3
28      29.0               6
29      30.0             473
...      ...             ...
1840  1841.0            1109
1841  1842.0               3
1842  1843.0  

In [89]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [90]:
time_period = zero_runs(df['PIX_VALUE_DIFF'].tolist())-1
time_period[0, :1] = 0
print("Number of PIX_VALUE_DIFF which value is '0': %s"%len(time_period))

Number of PIX_VALUE_DIFF which value is '0': 51


In [91]:
THRESHOLD = 0 # lag for more than 7 frames
filtered_time_period = time_period[np.where(np.diff(time_period)+1 >= THRESHOLD)[0]]

In [92]:
result = pd.DataFrame(columns=['START_FRAME', 'END_FRAME', 'FRAME_PERIOD'])

for i in range(0, len(filtered_time_period)):
    start_time = df.iloc[filtered_time_period[i][0]]['FRAME']
    end_time = df.iloc[filtered_time_period[i][1]]['FRAME']
    time_diff = end_time - start_time
    result = result.append({'START_FRAME': start_time, 'END_FRAME': end_time, 'FRAME_PERIOD': time_diff}, ignore_index=True)

In [93]:
print(result)

    START_FRAME  END_FRAME  FRAME_PERIOD
0           1.0        1.0           0.0
1         106.0      107.0           1.0
2         108.0      109.0           1.0
3         122.0      123.0           1.0
4         229.0      230.0           1.0
5         260.0      261.0           1.0
6         312.0      313.0           1.0
7         356.0      357.0           1.0
8         388.0      389.0           1.0
9         570.0      571.0           1.0
10        644.0      645.0           1.0
11        655.0      656.0           1.0
12        662.0      663.0           1.0
13        751.0      752.0           1.0
14        762.0      763.0           1.0
15        768.0      769.0           1.0
16        796.0      797.0           1.0
17        839.0      840.0           1.0
18        862.0      863.0           1.0
19        893.0      894.0           1.0
20        961.0      962.0           1.0
21       1002.0     1003.0           1.0
22       1007.0     1008.0           1.0
23       1013.0 

In [94]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool, TapTool
output_notebook()

Loading BokehJS ...

In [104]:
#df.loc[(df['PIX_VALUE_DIFF'] <= 100) & (df['PIX_VALUE_DIFF'] >= -100), 'PIX_VALUE_DIFF'] = 0
#df.loc[df['PIX_VALUE_DIFF']==0, 'DESC'] = 'Freeze frame'
#df.loc[df['PIX_VALUE_DIFF']!=0, 'DESC'] = 'Moving frame'

source = ColumnDataSource(data=dict(
    x = df['FRAME'],
    y = df['PIX_VALUE_DIFF'],
    #desc = df['DESC']
))

p = figure(title="Video Transition Analysis and Detection", width=1000, height=500, toolbar_location="below")

p.step('x', 'y', source=source, color="#778899", alpha=0.8,  mode="center")
cr = p.circle('x', 'y', source=source, radius=0.5, legend='Pixel Difference Value', fill_color="firebrick", hover_fill_color="red",
    line_color=None, hover_line_color="white", fill_alpha=0.2, hover_alpha=0.5)

hover = HoverTool(tooltips=[
    ("Frame No.", "@x"),
    ("Pixel Difference Value", "@y"),
    #("Desc", "@desc")
    ], 
    renderers=[cr])
p.add_tools(hover)

p.xaxis.axis_label = 'Frame'
p.yaxis.axis_label = 'Pixel Difference Value'
p.grid.grid_line_alpha=0.5

show(p)

In [105]:
p1 = figure(title="Distribution ratio",tools="save",
            background_fill_color="#E8DDCB")

x = np.linspace(0, 10, 50)
hist, edges = np.histogram(result['FRAME_PERIOD'], density=True, bins=50)

cdf = np.cumsum(hist)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
#p1.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p1.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p1.legend.location = "center_right"
p1.legend.background_fill_color = "darkgrey"
p1.xaxis.axis_label = 'Lag Frames'
p1.yaxis.axis_label = 'Distrobution Ratio'
show(gridplot(p1, ncols=2, plot_width=900, plot_height=400, toolbar_location=None))